<a href="https://colab.research.google.com/github/rakesh22230/Data-Mining-Lab/blob/main/Bangla_Fake_News_Training_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install one by one
import sys

print("Installing packages one by one...")

print("\n[1/4] Installing scikit-learn...")
!{sys.executable} -m pip install -q scikit-learn

print("[2/4] Installing pandas...")
!{sys.executable} -m pip install -q pandas

print("[3/4] Installing numpy...")
!{sys.executable} -m pip install -q numpy

print("[4/4] Installing scipy...")
!{sys.executable} -m pip install -q scipy

print("\n✅ All packages installed!")
print("📋 Run the next cell to test imports")

Installing packages one by one...

[1/4] Installing scikit-learn...
[2/4] Installing pandas...
[3/4] Installing numpy...
[4/4] Installing scipy...

✅ All packages installed!
📋 Run the next cell to test imports


In [ ]:
# Test imports
import sklearn
import pandas as pd
import numpy as np
from scipy.sparse import hstack

print("✅ All imports successful!")
print(f"   scikit-learn: {sklearn.__version__}")
print(f"   pandas: {pd.__version__}")
print(f"   numpy: {np.__version__}")

✅ All imports successful!
   scikit-learn: 1.6.1
   pandas: 2.2.2
   numpy: 2.0.2


In [ ]:
import os

print("=" * 80)
print("📁 CHECKING DATASET FILES")
print("=" * 80)

# Define the base path for your files in Google Drive
base_path = '/content/drive/MyDrive/Colab Notebooks/'

# Check for required files
required_files = ['train_cleaned.csv', 'val_cleaned.csv', 'test_cleaned.csv']
missing_files = []

print(f"\n📂 Looking for files in: {base_path}...")
for file_name in required_files:
    file_path = os.path.join(base_path, file_name)
    if os.path.exists(file_path):
        size_mb = os.path.getsize(file_path) / (1024 * 1024)
        print(f"   ✅ {file_name} - {size_mb:.2f} MB")
    else:
        print(f"   ❌ {file_name} - NOT FOUND")
        missing_files.append(file_name)

print("=" * 80)

if missing_files:
    print(f"\n❌ ERROR: Missing {len(missing_files)} file(s) in {base_path}:")
    for f in missing_files:
        print(f"   - {f}")
    print("\n💡 Instructions:")
    print(f"   Ensure these files are present in your Google Drive folder: {base_path}")
    print("   If the path is incorrect, please let me know the correct path.")
    print("   After confirming files are there, re-run this cell.")
else:
    print("\n✅ ALL FILES PRESENT!")
    print("🚀 You can now proceed to CELL 3!")

📁 CHECKING DATASET FILES

📂 Looking for files in: /content/drive/MyDrive/Colab Notebooks/...
   ✅ train_cleaned.csv - 199.72 MB
   ✅ val_cleaned.csv - 42.69 MB
   ✅ test_cleaned.csv - 42.81 MB

✅ ALL FILES PRESENT!
🚀 You can now proceed to CELL 3!


In [ ]:
import pandas as pd
import os # Import os module to join paths

print("=" * 80)
print("📊 LOADING DATASET")
print("=" * 80)

# Define the base path for your files (same as in the file checking cell)
base_path = '/content/drive/MyDrive/Colab Notebooks/'

# Load datasets using the full path
train_df = pd.read_csv(os.path.join(base_path, 'train_cleaned.csv'), encoding='utf-8')
val_df = pd.read_csv(os.path.join(base_path, 'val_cleaned.csv'), encoding='utf-8')
test_df = pd.read_csv(os.path.join(base_path, 'test_cleaned.csv'), encoding='utf-8')

print(f"\n✅ Training set: {train_df.shape[0]:,} samples × {train_df.shape[1]} columns")
print(f"✅ Validation set: {val_df.shape[0]:,} samples × {val_df.shape[1]} columns")
print(f"✅ Test set: {test_df.shape[0]:,} samples × {test_df.shape[1]} columns")

print(f"\n📋 Columns: {train_df.columns.tolist()}")

# Show sample
print(f"\n📰 Sample Data (first 2 rows):")
print(train_df.head(2))

# Label distribution
print(f"\n📊 Original Label Distribution:")
label_counts = train_df['Label'].value_counts().sort_index()
for label, count in label_counts.items():
    print(f"   Label {label}: {count:,} samples ({count/len(train_df)*100:.1f}%)")

print("\n✅ Data loaded successfully!")

📊 LOADING DATASET

✅ Training set: 42,380 samples × 3 columns
✅ Validation set: 9,082 samples × 3 columns
✅ Test set: 9,082 samples × 3 columns

📋 Columns: ['Headline', 'Content', 'Label']

📰 Sample Data (first 2 rows):
                                Headline  \
0          কলকাতায় বোমা বিস্ফোরণে নিহত ১   
1  ১৫ ঘণ্টা পর ময়মনসিংহের পথে ট্রেন চালু   

                                             Content  Label  
0  ভারতের পশ্চিমবঙ্গের রাজধানী কলকাতায় একটি বাজার...      3  
1  ময়মনিসংহ স্টেশনের সুপারিনটেনডেন্ট জহিরুল হক জা...      2  

📊 Original Label Distribution:
   Label 0: 1,524 samples (3.6%)
   Label 1: 2,112 samples (5.0%)
   Label 2: 4,629 samples (10.9%)
   Label 3: 34,115 samples (80.5%)

✅ Data loaded successfully!


In [ ]:
import re
import numpy as np

print("=" * 80)
print("🔧 DEFINING PREPROCESSING FUNCTIONS")
print("=" * 80)

# Label mapping function
def map_labels(label):
    """
    Binary classification:
    - Label 1 (Authentic) → 1 (Real News)
    - Labels 0, 2, 3 (Fake, Satire, Clickbait) → 0 (Fake News)
    """
    return 1 if label == 1 else 0

# Text cleaning function
def clean_bangla_text(text):
    """
    Enhanced Bangla text cleaning:
    - Keeps Bengali numbers (০-৯) for dates, scores
    - Keeps Bengali punctuation (। ?)
    - Removes only English letters
    - Preserves maximum context
    """
    if pd.isna(text):
        return ""

    # Remove English letters only
    text = re.sub(r'[a-zA-Z]', ' ', text)

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text)

    # Keep Bengali chars + numbers + punctuation
    text = re.sub(r'[^\u0980-\u09FF\s০-৯\u0964\u0965]', '', text)

    return text.strip()

# Minimal stopwords list
bangla_stopwords = ['এবং', 'কিন্তু', 'যে', 'এই']

def remove_stopwords(text):
    """Remove only the most common stopwords"""
    words = text.split()
    filtered = [word for word in words if word not in bangla_stopwords]
    return ' '.join(filtered)

# Statistical feature extraction
def extract_statistical_features(texts):
    """
    Extract statistical features that help identify fake news:
    - Word count, character count, average word length
    - Punctuation patterns, question marks, exclamations
    - Long and short words, numbers
    """
    features = []
    for text in texts:
        words = text.split()
        features.append([
            len(words),                              # Word count
            len(text),                               # Character count
            len(words) / (len(text) + 1),           # Average word length
            text.count('।'),                         # Bengali punctuation
            text.count('?'),                         # Question marks
            text.count('!'),                         # Exclamations
            len([w for w in words if len(w) > 10]), # Long words (>10 chars)
            len([w for w in words if len(w) < 3]),  # Short words (<3 chars)
            text.count(' ০') + text.count(' ১') + text.count(' ২'),  # Number mentions
        ])
    return np.array(features)

print("✅ All preprocessing functions defined!")
print("\n📋 Functions:")
print("   ✅ map_labels() - Convert to binary (0=Fake, 1=Real)")
print("   ✅ clean_bangla_text() - Clean and normalize text")
print("   ✅ remove_stopwords() - Remove common words")
print("   ✅ extract_statistical_features() - Extract 9 statistical features")

🔧 DEFINING PREPROCESSING FUNCTIONS
✅ All preprocessing functions defined!

📋 Functions:
   ✅ map_labels() - Convert to binary (0=Fake, 1=Real)
   ✅ clean_bangla_text() - Clean and normalize text
   ✅ remove_stopwords() - Remove common words
   ✅ extract_statistical_features() - Extract 9 statistical features


In [ ]:
print("=" * 80)
print("🔄 PREPROCESSING DATA")
print("=" * 80)

# Step 1: Map labels to binary
print("\n[1/3] Mapping labels to binary...")
train_df['Label_Binary'] = train_df['Label'].apply(map_labels)
val_df['Label_Binary'] = val_df['Label'].apply(map_labels)
test_df['Label_Binary'] = test_df['Label'].apply(map_labels)

fake_count = (train_df['Label_Binary'] == 0).sum()
real_count = (train_df['Label_Binary'] == 1).sum()
print(f"   Fake News (0): {fake_count:,} samples ({fake_count/len(train_df)*100:.1f}%)")
print(f"   Real News (1): {real_count:,} samples ({real_count/len(train_df)*100:.1f}%)")

# Step 2: Combine Headline and Content
print("\n[2/3] Combining Headline + Content...")
train_df['full_text'] = train_df['Headline'].fillna('') + ' ' + train_df['Content'].fillna('')
val_df['full_text'] = val_df['Headline'].fillna('') + ' ' + val_df['Content'].fillna('')
test_df['full_text'] = test_df['Headline'].fillna('') + ' ' + test_df['Content'].fillna('')

# Step 3: Clean and remove stopwords
print("\n[3/3] Cleaning text and removing stopwords...")
train_df['cleaned_text'] = train_df['full_text'].apply(clean_bangla_text).apply(remove_stopwords)
val_df['cleaned_text'] = val_df['full_text'].apply(clean_bangla_text).apply(remove_stopwords)
test_df['cleaned_text'] = test_df['full_text'].apply(clean_bangla_text).apply(remove_stopwords)

# Remove very short texts
initial_train = len(train_df)
initial_test = len(test_df)
train_df = train_df[train_df['cleaned_text'].str.len() > 10]
test_df = test_df[test_df['cleaned_text'].str.len() > 10]

print(f"   Removed {initial_train - len(train_df)} short training samples")
print(f"   Removed {initial_test - len(test_df)} short test samples")

print(f"\n✅ Preprocessing complete!")
print(f"   Final training samples: {len(train_df):,}")
print(f"   Final test samples: {len(test_df):,}")

# Show sample cleaned text
print(f"\n📝 Sample Cleaned Text:")
sample_original = train_df['full_text'].iloc[0][:100]
sample_cleaned = train_df['cleaned_text'].iloc[0][:100]
print(f"   Original: {sample_original}...")
print(f"   Cleaned:  {sample_cleaned}...")

🔄 PREPROCESSING DATA

[1/3] Mapping labels to binary...
   Fake News (0): 40,268 samples (95.0%)
   Real News (1): 2,112 samples (5.0%)

[2/3] Combining Headline + Content...

[3/3] Cleaning text and removing stopwords...
   Removed 5 short training samples
   Removed 2 short test samples

✅ Preprocessing complete!
   Final training samples: 42,375
   Final test samples: 9,080

📝 Sample Cleaned Text:
   Original: কলকাতায় বোমা বিস্ফোরণে নিহত ১ ভারতের পশ্চিমবঙ্গের রাজধানী কলকাতায় একটি বাজারে বোমা বিস্ফোরণে আট বছরে...
   Cleaned:  কলকাতায় বোমা বিস্ফোরণে নিহত ১ ভারতের পশ্চিমবঙ্গের রাজধানী কলকাতায় একটি বাজারে বোমা বিস্ফোরণে আট বছরে...


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

print("=" * 80)
print("🔍 FEATURE EXTRACTION")
print("=" * 80)

# Prepare data
X_train_text = train_df['cleaned_text']
y_train = train_df['Label_Binary']
X_test_text = test_df['cleaned_text']
y_test = test_df['Label_Binary']

# Step 1: TF-IDF Features
print("\n[1/3] Extracting TF-IDF features...")
print("   Parameters:")
print("   - Max features: 12,000")
print("   - N-grams: 1-3 (unigrams, bigrams, trigrams)")
print("   - Min document frequency: 2")
print("   - Max document frequency: 85%")

tfidf = TfidfVectorizer(
    max_features=12000,
    ngram_range=(1, 3),
    min_df=2,
    max_df=0.85,
    sublinear_tf=True,
    strip_accents=None,
    lowercase=False
)

X_train_tfidf = tfidf.fit_transform(X_train_text)
X_test_tfidf = tfidf.transform(X_test_text)

print(f"   ✅ TF-IDF shape: {X_train_tfidf.shape}")
print(f"   ✅ Vocabulary size: {len(tfidf.vocabulary_):,} words")

# Step 2: Statistical Features
print("\n[2/3] Extracting statistical features...")
train_stats = extract_statistical_features(X_train_text)
test_stats = extract_statistical_features(X_test_text)

print(f"   ✅ Statistical features shape: {train_stats.shape}")
print(f"   ✅ Features: 9 (word count, char count, punctuation, etc.)")

# Step 3: Combine Features
print("\n[3/3] Combining TF-IDF + Statistical features...")
X_train_combined = hstack([X_train_tfidf, train_stats])
X_test_combined = hstack([X_test_tfidf, test_stats])

print(f"   ✅ Final combined features: {X_train_combined.shape}")
print(f"   ✅ Total features: {X_train_combined.shape[1]:,}")

print("\n✅ Feature extraction complete!")

🔍 FEATURE EXTRACTION

[1/3] Extracting TF-IDF features...
   Parameters:
   - Max features: 12,000
   - N-grams: 1-3 (unigrams, bigrams, trigrams)
   - Min document frequency: 2
   - Max document frequency: 85%
   ✅ TF-IDF shape: (42375, 12000)
   ✅ Vocabulary size: 12,000 words

[2/3] Extracting statistical features...
   ✅ Statistical features shape: (42375, 9)
   ✅ Features: 9 (word count, char count, punctuation, etc.)

[3/3] Combining TF-IDF + Statistical features...
   ✅ Final combined features: (42375, 12009)
   ✅ Total features: 12,009

✅ Feature extraction complete!


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score
import time

print("=" * 80)
print("🤖 TRAINING MACHINE LEARNING MODELS")
print("=" * 80)

models = {}

# Model 1: Naive Bayes
print("\n[Model 1/5] Naive Bayes...")
start = time.time()
nb_model = MultinomialNB(alpha=0.5)
nb_model.fit(X_train_combined, y_train)
nb_pred = nb_model.predict(X_test_combined)
nb_acc = accuracy_score(y_test, nb_pred)
models['Naive Bayes'] = (nb_model, nb_acc, nb_pred)
print(f"   ✅ Accuracy: {nb_acc*100:.2f}% (Time: {time.time()-start:.1f}s)")

# Model 2: Logistic Regression
print("\n[Model 2/5] Logistic Regression...")
start = time.time()
lr_model = LogisticRegression(
    max_iter=3000,
    C=2.0,
    random_state=42,
    solver='saga',
    class_weight='balanced',
    n_jobs=-1
)
lr_model.fit(X_train_combined, y_train)
lr_pred = lr_model.predict(X_test_combined)
lr_acc = accuracy_score(y_test, lr_pred)
models['Logistic Regression'] = (lr_model, lr_acc, lr_pred)
print(f"   ✅ Accuracy: {lr_acc*100:.2f}% (Time: {time.time()-start:.1f}s)")

# Model 3: Linear SVM (FAST)
print("\n[Model 3/5] Linear SVM (LinearSVC)...")
start = time.time()
svm_model = LinearSVC(
    C=1.5,
    class_weight='balanced',
    max_iter=2000,
    random_state=42
)
svm_model.fit(X_train_combined, y_train)
svm_pred = svm_model.predict(X_test_combined)
svm_acc = accuracy_score(y_test, svm_pred)
models['Linear SVM'] = (svm_model, svm_acc, svm_pred)
print(f"   ✅ Accuracy: {svm_acc*100:.2f}% (Time: {time.time()-start:.1f}s)")

# Model 4: Random Forest
print("\n[Model 4/5] Random Forest...")
start = time.time()
rf_model = RandomForestClassifier(
    n_estimators=300,
    max_depth=60,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'
)
rf_model.fit(X_train_combined, y_train)
rf_pred = rf_model.predict(X_test_combined)
rf_acc = accuracy_score(y_test, rf_pred)
models['Random Forest'] = (rf_model, rf_acc, rf_pred)
print(f"   ✅ Accuracy: {rf_acc*100:.2f}% (Time: {time.time()-start:.1f}s)")

# Model 5: Ensemble (Hard Voting)
print("\n[Model 5/5] Ensemble (Hard Voting)...")
print("   Combining: Logistic Regression + Linear SVM + Random Forest")
start = time.time()
ensemble_model = VotingClassifier(
    estimators=[
        ('lr', lr_model),
        ('svm', svm_model),
        ('rf', rf_model)
    ],
    voting='hard',
    n_jobs=-1
)
ensemble_model.fit(X_train_combined, y_train)
ensemble_pred = ensemble_model.predict(X_test_combined)
ensemble_acc = accuracy_score(y_test, ensemble_pred)
models['Ensemble'] = (ensemble_model, ensemble_acc, ensemble_pred)
print(f"   ✅ Accuracy: {ensemble_acc*100:.2f}% (Time: {time.time()-start:.1f}s)")

# Display all results
print("\n" + "=" * 80)
print("📊 MODEL COMPARISON (sorted by accuracy)")
print("=" * 80)
for name, (_, acc, _) in sorted(models.items(), key=lambda x: x[1][1], reverse=True):
    print(f"   {name:25s}: {acc*100:.2f}%")

print("\n✅ All models trained successfully!")


🤖 TRAINING MACHINE LEARNING MODELS

[Model 1/5] Naive Bayes...
   ✅ Accuracy: 92.22% (Time: 0.4s)

[Model 2/5] Logistic Regression...
   ✅ Accuracy: 68.06% (Time: 366.4s)

[Model 3/5] Linear SVM (LinearSVC)...
   ✅ Accuracy: 90.08% (Time: 20.3s)

[Model 4/5] Random Forest...
   ✅ Accuracy: 94.03% (Time: 153.2s)

[Model 5/5] Ensemble (Hard Voting)...
   Combining: Logistic Regression + Linear SVM + Random Forest
   ✅ Accuracy: 92.04% (Time: 530.2s)

📊 MODEL COMPARISON (sorted by accuracy)
   Random Forest            : 94.03%
   Naive Bayes              : 92.22%
   Ensemble                 : 92.04%
   Linear SVM               : 90.08%
   Logistic Regression      : 68.06%

✅ All models trained successfully!


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

print("=" * 80)
print("🏆 SELECTING BEST MODEL & EVALUATION")
print("=" * 80)

# Select best model
best_model_name = max(models, key=lambda k: models[k][1])
best_model, best_accuracy, best_predictions = models[best_model_name]

print(f"\n🏆 Best Model: {best_model_name}")
print(f"🏆 Test Accuracy: {best_accuracy*100:.2f}%")

# Cross-validation
print(f"\n⏳ Running 5-fold Cross-Validation...")
print(f"   (This validates model stability across different data splits)")
cv_scores = cross_val_score(
    best_model,
    X_train_combined,
    y_train,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=0
)

print(f"\n✅ Cross-Validation Results:")
print(f"   Fold 1: {cv_scores[0]*100:.2f}%")
print(f"   Fold 2: {cv_scores[1]*100:.2f}%")
print(f"   Fold 3: {cv_scores[2]*100:.2f}%")
print(f"   Fold 4: {cv_scores[3]*100:.2f}%")
print(f"   Fold 5: {cv_scores[4]*100:.2f}%")
print(f"   ─────────────────────")
print(f"   Mean:   {cv_scores.mean()*100:.2f}%")
print(f"   Std:    ±{cv_scores.std()*100:.2f}%")
print(f"   Range:  {cv_scores.min()*100:.2f}% - {cv_scores.max()*100:.2f}%")

# Detailed classification report
print(f"\n" + "=" * 80)
print("📈 DETAILED CLASSIFICATION REPORT")
print("=" * 80)
print(classification_report(
    y_test,
    best_predictions,
    labels=[0, 1],
    target_names=['Fake News', 'Real News'],
    digits=4
))

# Confusion Matrix
print("=" * 80)
print("📊 CONFUSION MATRIX")
print("=" * 80)
cm = confusion_matrix(y_test, best_predictions, labels=[0, 1])
print(f"\n                    Predicted")
print(f"                 Fake      Real")
print(f"   Actual Fake   {cm[0][0]:6d}    {cm[0][1]:6d}")
print(f"          Real   {cm[1][0]:6d}    {cm[1][1]:6d}")

# Calculate detailed metrics
tn, fp, fn, tp = cm[0][0], cm[0][1], cm[1][0], cm[1][1]

fake_precision = tn / (tn + fn) if (tn + fn) > 0 else 0
fake_recall = tn / (tn + fp) if (tn + fp) > 0 else 0
fake_f1 = 2 * (fake_precision * fake_recall) / (fake_precision + fake_recall) if (fake_precision + fake_recall) > 0 else 0

real_precision = tp / (fp + tp) if (fp + tp) > 0 else 0
real_recall = tp / (fn + tp) if (fn + tp) > 0 else 0
real_f1 = 2 * (real_precision * real_recall) / (real_precision + real_recall) if (real_precision + real_recall) > 0 else 0

print(f"\n" + "=" * 80)
print("📊 KEY PERFORMANCE METRICS")
print("=" * 80)

print(f"\n🚨 FAKE NEWS DETECTION:")
print(f"   Precision: {fake_precision*100:.2f}% (How many detected fakes are actually fake)")
print(f"   Recall:    {fake_recall*100:.2f}% (How many actual fakes we detected)")
print(f"   F1-Score:  {fake_f1*100:.2f}%")

print(f"\n✅ REAL NEWS DETECTION:")
print(f"   Precision: {real_precision*100:.2f}% (How many detected reals are actually real)")
print(f"   Recall:    {real_recall*100:.2f}% (How many actual reals we detected)")
print(f"   F1-Score:  {real_f1*100:.2f}%")

print(f"\n⚠️ ERROR RATES:")
print(f"   False Positive: {fp/(tn+fp)*100:.2f}% (Real news wrongly marked as fake)")
print(f"   False Negative: {fn/(fn+tp)*100:.2f}% (Fake news wrongly marked as real)")

print("\n✅ Evaluation complete!")

🏆 SELECTING BEST MODEL & EVALUATION

🏆 Best Model: Random Forest
🏆 Test Accuracy: 94.03%

⏳ Running 5-fold Cross-Validation...
   (This validates model stability across different data splits)

✅ Cross-Validation Results:
   Fold 1: 94.55%
   Fold 2: 94.31%
   Fold 3: 94.11%
   Fold 4: 94.23%
   Fold 5: 94.06%
   ─────────────────────
   Mean:   94.25%
   Std:    ±0.17%
   Range:  94.06% - 94.55%

📈 DETAILED CLASSIFICATION REPORT
              precision    recall  f1-score   support

   Fake News     0.9563    0.9822    0.9690      8636
   Real News     0.2667    0.1261    0.1713       444

    accuracy                         0.9403      9080
   macro avg     0.6115    0.5541    0.5701      9080
weighted avg     0.9225    0.9403    0.9300      9080

📊 CONFUSION MATRIX

                    Predicted
                 Fake      Real
   Actual Fake     8482       154
          Real      388        56

📊 KEY PERFORMANCE METRICS

🚨 FAKE NEWS DETECTION:
   Precision: 95.63% (How many detected

In [ ]:
import pickle

print("=" * 80)
print("💾 SAVING TRAINED MODELS")
print("=" * 80)

# Save best model
print("\n[1/4] Saving best model...")
with open('fake_news_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)
print("   ✅ fake_news_model.pkl")

# Save TF-IDF vectorizer
print("\n[2/4] Saving TF-IDF vectorizer...")
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)
print("   ✅ tfidf_vectorizer.pkl")

# Save feature configuration
print("\n[3/4] Saving feature configuration...")
feature_config = {
    'extract_stats': extract_statistical_features,
    'clean_text': clean_bangla_text,
    'remove_stopwords': remove_stopwords,
    'stopwords': bangla_stopwords
}
with open('feature_config.pkl', 'wb') as f:
    pickle.dump(feature_config, f)
print("   ✅ feature_config.pkl")

# Save comprehensive metadata
print("\n[4/4] Saving model metadata...")
metadata = {
    'model_name': best_model_name,
    'accuracy': float(best_accuracy),
    'cv_mean': float(cv_scores.mean()),
    'cv_std': float(cv_scores.std()),
    'training_samples': len(train_df),
    'test_samples': len(test_df),
    'vocabulary_size': len(tfidf.vocabulary_),
    'fake_precision': float(fake_precision),
    'fake_recall': float(fake_recall),
    'fake_f1': float(fake_f1),
    'real_precision': float(real_precision),
    'real_recall': float(real_recall),
    'real_f1': float(real_f1),
    'false_positive_rate': float(fp/(tn+fp)),
    'false_negative_rate': float(fn/(fn+tp)),
    'ngram_range': '1-3',
    'max_features': 12000,
    'has_statistical_features': True
}
with open('model_metadata.pkl', 'wb') as f:
    pickle.dump(metadata, f)
print("   ✅ model_metadata.pkl")

print("\n" + "=" * 80)
print("✅ ALL MODEL FILES SAVED SUCCESSFULLY!")
print("=" * 80)

# List saved files with sizes
print("\n📁 Saved Files:")
import os
for filename in ['fake_news_model.pkl', 'tfidf_vectorizer.pkl', 'feature_config.pkl', 'model_metadata.pkl']:
    if os.path.exists(filename):
        size = os.path.getsize(filename) / (1024 * 1024)  # MB
        print(f"   ✅ {filename:25s} - {size:.2f} MB")

💾 SAVING TRAINED MODELS

[1/4] Saving best model...
   ✅ fake_news_model.pkl

[2/4] Saving TF-IDF vectorizer...
   ✅ tfidf_vectorizer.pkl

[3/4] Saving feature configuration...
   ✅ feature_config.pkl

[4/4] Saving model metadata...
   ✅ model_metadata.pkl

✅ ALL MODEL FILES SAVED SUCCESSFULLY!

📁 Saved Files:
   ✅ fake_news_model.pkl       - 73.95 MB
   ✅ tfidf_vectorizer.pkl      - 0.51 MB
   ✅ feature_config.pkl        - 0.00 MB
   ✅ model_metadata.pkl        - 0.00 MB


In [ ]:
from google.colab import files
import os

print("=" * 80)
print("⬇️ DOWNLOAD TRAINED MODELS TO YOUR COMPUTER")
print("=" * 80)

print("\n📦 Preparing to download 4 model files...")
print("   These will be saved to your Downloads folder")
print("\n⏳ Downloading (this may take 30-60 seconds)...\n")

# Download all model files
try:
    files.download('fake_news_model.pkl')
    print("   ✅ fake_news_model.pkl downloaded")

    files.download('tfidf_vectorizer.pkl')
    print("   ✅ tfidf_vectorizer.pkl downloaded")

    files.download('feature_config.pkl')
    print("   ✅ feature_config.pkl downloaded")

    files.download('model_metadata.pkl')
    print("   ✅ model_metadata.pkl downloaded")

    print("\n✅ ALL FILES DOWNLOADED SUCCESSFULLY!")

except Exception as e:
    print(f"\n⚠️ Download error: {e}")
    print("Alternative: Click folder icon on left → Right-click each .pkl file → Download")

print("\n" + "=" * 80)
print("📋 NEXT STEPS:")
print("=" * 80)
print("1. Check your Downloads folder for 4 .pkl files")
print("2. Copy all 4 files to your project's 'models/' folder")
print("3. Update your app.py with the new version")
print("4. Run: python app.py")
print("5. Open: http://localhost:5000")
print("=" * 80)

⬇️ DOWNLOAD TRAINED MODELS TO YOUR COMPUTER

📦 Preparing to download 4 model files...
   These will be saved to your Downloads folder

⏳ Downloading (this may take 30-60 seconds)...



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

   ✅ fake_news_model.pkl downloaded


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

   ✅ feature_config.pkl downloaded


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

   ✅ model_metadata.pkl downloaded

✅ ALL FILES DOWNLOADED SUCCESSFULLY!

📋 NEXT STEPS:
1. Check your Downloads folder for 4 .pkl files
2. Copy all 4 files to your project's 'models/' folder
3. Update your app.py with the new version
4. Run: python app.py
5. Open: http://localhost:5000


In [ ]:
print("\n" + "=" * 80)
print("🎉 TRAINING COMPLETE - FINAL SUMMARY")
print("=" * 80)

print(f"\n📊 MODEL PERFORMANCE:")
print(f"   Model Type:     {best_model_name}")
print(f"   Test Accuracy:  {best_accuracy*100:.2f}%")
print(f"   CV Accuracy:    {cv_scores.mean()*100:.2f}% (±{cv_scores.std()*100:.2f}%)")
print(f"   Fake Detection: {fake_recall*100:.2f}%")
print(f"   Real Detection: {real_recall*100:.2f}%")

print(f"\n📈 TRAINING DATA:")
print(f"   Training Samples: {len(train_df):,}")
print(f"   Test Samples:     {len(test_df):,}")
print(f"   Vocabulary:       {len(tfidf.vocabulary_):,} words")
print(f"   Features:         TF-IDF (12K) + Statistical (9)")

print(f"\n🎯 MODEL CHARACTERISTICS:")
print(f"   ✅ Enhanced preprocessing (minimal stopwords)")
print(f"   ✅ TF-IDF with trigrams (1-3 n-grams)")
print(f"   ✅ Statistical features included")
print(f"   ✅ {'Ensemble voting' if 'Ensemble' in best_model_name else 'Single model'}")
print(f"   ✅ Cross-validated for stability")
print(f"   ✅ Balanced class weights")

print(f"\n💾 SAVED FILES:")
print(f"   ✅ fake_news_model.pkl")
print(f"   ✅ tfidf_vectorizer.pkl")
print(f"   ✅ feature_config.pkl")
print(f"   ✅ model_metadata.pkl")

print("\n" + "=" * 80)
print("🧪 QUICK TEST - Let's predict a sample!")
print("=" * 80)

# Test with a sample
test_sample = test_df['cleaned_text'].iloc[0]
test_label = y_test.iloc[0]

# Extract features
test_tfidf = tfidf.transform([test_sample])
test_stats = extract_statistical_features([test_sample])
test_features = hstack([test_tfidf, test_stats])

# Predict
prediction = best_model.predict(test_features)[0]
proba = best_model.predict_proba(test_features)[0]

print(f"\n📰 Sample News (first 150 chars):")
print(f"   {test_df['full_text'].iloc[0][:150]}...")
print(f"\n🔍 Prediction:")
print(f"   Predicted: {'Real News' if prediction == 1 else 'Fake News'}")
print(f"   Actual:    {'Real News' if test_label == 1 else 'Fake News'}")
print(f"   Correct:   {'✅ YES' if prediction == test_label else '❌ NO'}")
print(f"   Confidence: {max(proba)*100:.2f}%")
print(f"   Probabilities: Fake={proba[0]*100:.2f}%, Real={proba[1]*100:.2f}%")

print("\n" + "=" * 80)
print("✅ ALL DONE! Your model is ready to use!")
print("=" * 80)


🎉 TRAINING COMPLETE - FINAL SUMMARY

📊 MODEL PERFORMANCE:
   Model Type:     Random Forest
   Test Accuracy:  94.03%
   CV Accuracy:    94.25% (±0.17%)
   Fake Detection: 98.22%
   Real Detection: 12.61%

📈 TRAINING DATA:
   Training Samples: 42,375
   Test Samples:     9,080
   Vocabulary:       12,000 words
   Features:         TF-IDF (12K) + Statistical (9)

🎯 MODEL CHARACTERISTICS:
   ✅ Enhanced preprocessing (minimal stopwords)
   ✅ TF-IDF with trigrams (1-3 n-grams)
   ✅ Statistical features included
   ✅ Single model
   ✅ Cross-validated for stability
   ✅ Balanced class weights

💾 SAVED FILES:
   ✅ fake_news_model.pkl
   ✅ tfidf_vectorizer.pkl
   ✅ feature_config.pkl
   ✅ model_metadata.pkl

🧪 QUICK TEST - Let's predict a sample!

📰 Sample News (first 150 chars):
   জিডিপিতে বীমার অবদান মাত্র ০.৯ শতাংশ আর্থিক খাতগুলোর মধ্যে বীমা গুরুত্বপূর্ণ হলেও মোট দেশজ উৎপাদনে (জিডিপি) বীমার অবদান মাত্র ০.৯ শতাংশ। তবে বীমা উন্ন...

🔍 Prediction:
   Predicted: Fake News
   Actual:    Fake Ne